In [1]:
import openmeteo_requests as omr
import pandas as pd 
import requests_cache
from retry_requests import retry
from geopy.geocoders import Nominatim

In [12]:

def getcoordinates(userlocation):
    """add docstring"""
    geolocator=Nominatim(user_agent="Dress for the Weather")
    location=geolocator.geocode(userlocation)
    coordinates=[location.latitude, location.longitude]
    #print(coordinates) # don't use print inside the function, print the return
    return coordinates
 

In [15]:
# test getcoordinates function
location = "Seattle, WA"
lat, lon = getcoordinates(location)
print(f"{location}: Latitude: {lat}, Longitude: {lon}")

Seattle, WA: Latitude: 47.6038321, Longitude: -122.330062


In [ ]:

def getweather(lat, lon): # get weather data for given lat, lon
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = omr.Client(session = retry_session)

    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m", 
        "forecast_days": 1,
        "temperature_unit": "fahrenheit",
        "wind_speed_unit": "mph",
        "precipitation_unit": "inch",
        "current": ["temperature_2m"],
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation: {response.Elevation()} m asl")
    print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")
    #get rid of print statements
    # You could also return elevation and timezone difference if needed

    # Process hourly data. The order of variables needs to be the same as requested.
    current = response.Current()
    current_temperature_2m = current.Variables(0).Value()

    #hourly_data = {"date": pd.Timestamp.today}

    #hourly_data["temperature_2m"] = hourly

    #hourly_dataframe = pd.DataFrame(data = hourly_data)

    # Why not use response.Hourly().DataFrame() directly?
    return round(current_temperature_2m, 1)
    #print("\nHourly data\n", current_temperature_2m)


In [16]:
# test getweather
current_temp = getweather(lat, lon)
print(f"Current Temperature in {location}: {current_temp}°F")

Coordinates: 47.59556198120117°N -122.32443237304688°E
Elevation: 40.0 m asl
Timezone difference to GMT+0: 0s
Current Temperature in Seattle, WA: 56.3°F


In [ ]:
def weather(temperature):
    if temperature > 80:
        print("Shorts and a t-shirt")
    elif 60 <= temperature <= 70:
        print("Shorts and a t-shirt or a light jacket")
    elif 50 <= temperature < 60:
        print("pants and a longsleeve")
    elif 40 <= temperature < 50:
        print("pants and a longsleeve with a light jacket")

In [ ]:
# test weather using current_temp
???